# PyTao plotting with Bokeh

PyTao supports plotting directly from the notebook, without a separate (X11) plot window.

PyTao provides two backends:
* Bokeh (with interactive plotting support)
* Matplotlib 

When plotting is enabled, PyTao will automatically select the best available backend.


The plotting backend may be specified explicitly, as we will do in this notebook in order to show off
this backend's functionality.

---
## Tao setup

In [1]:
from pytao import SubprocessTao, Tao

In [2]:
init_file = "$ACC_ROOT_DIR/bmad-doc/tao_examples/optics_matching/tao.init"

tao = Tao(init_file=init_file, plot="bokeh")

## Shape setup

Let's update Tao's plotting shapes first, so that we see label names in the layout and floor plan.
Customizing this in the Bmad init files is an alternative to doing this in PyTao.

In [3]:
_ = tao.update_plot_shapes("quadrupole", type_label="name", layout=True, floor=True)

## The floor plan

In [4]:
tao.plot("floor_plan")

<!-- Detected Jupyter. -->

Loading BokehJS ...

In [5]:
tao.bokeh.layout_template = "lat_layout"

## Single data plots

In [6]:
tao.plot("dispersion")

## Plot fields

In [7]:
tao.cmd("set var quad[1]|model = -5")
tao.plot_field("Q1")

## Stacked plots

In [8]:
tao.plot(["alpha", "beta"])

In [9]:
tao.plot(["alpha", "beta"], include_layout=False)

## Gridded plots

In [10]:
tao.plot(["beta", "dispersion", "orbit"], grid=(2, 2), width=350, height=100, layout_height=50)

## Saving plots

The parameter `save` makes it convenient to simultaneously display and save the plot to a file.

In [11]:
tao.plot("beta", save="beta", include_layout=False)

## Change defaults

In [12]:
tao.bokeh.configure(
    width=500,
    height=200,
    layout_height=25,
    palette="Magma256",
)

{'width': 500,
 'height': 200,
 'stacked_height': 200,
 'layout_height': 25,
 'show_bokeh_logo': False,
 'palette': 'Magma256',
 'tools': 'pan,wheel_zoom,box_zoom,reset,hover,crosshair',
 'grid_toolbar_location': 'right',
 'lattice_layout_tools': 'pan,wheel_zoom,box_zoom,reset,hover,crosshair',
 'floor_plan_tools': 'pan,wheel_zoom,box_zoom,reset,hover,crosshair',
 'floor_plan_annotate_elements': True,
 'layout_font_size': '0.75em',
 'floor_plan_font_size': '0.75em',
 'limit_scale_factor': 1.01,
 'max_data_points': 10000,
 'variables_per_row': 2,
 'show_sliders': True,
 'line_width_scale': 0.5,
 'floor_line_width_scale': 0.5}

In [13]:
tao.plot("beta")

## Accessing plot data

In [14]:
graphs, app = tao.last_plot

In [15]:
graphs[0].curves[0].line.xs[:10]

[0.0,
 0.0064516129032258064,
 0.012903225806451613,
 0.019354838709677417,
 0.025806451612903226,
 0.03225806451612903,
 0.038709677419354833,
 0.04516129032258065,
 0.05161290322580645,
 0.058064516129032254]

## Advanced plotting settings

`TaoGraphSettings` may be used to customize per-graph settings in single or gridded plots.

Since each graph has its own settings, we specify a list of `TaoGraphSettings`.
`TaoGraphSettings` includes customization of titles, legends, scales, per-axis settings (with `TaoAxisSettings`), and so on.  

For example, to change the title of a plot, one could use:
`TaoGraphSettings(title="something")` - or equivalently a custom Tao command can be sent with `TaoGraphSettings(commands=["set graph {graph} title = something"])`.

See `TaoGraphSettings` documentation for further information on what may be customized. Not all settings will be supported by PyTao's plotting backends.

In [16]:
from pytao.plotting import TaoGraphSettings, TaoAxisSettings

# Let's use SubprocessTao to make an independent Tao instance in a subprocess.
# Now we can use `tao` from above (`optics_matching`) and `erl` here simultaneously.
erl = SubprocessTao(
    init_file="$ACC_ROOT_DIR/bmad-doc/tao_examples/erl/tao.init", 
    plot="bokeh", 
    # The ERL example startup file customizes plots in a way incompatible with pytao's external plotting.
    # Set "nostartup=True" to avoid this.
    nostartup=True,
)

erl.cmds(
    [
        "set global track_type = beam",
        "set var r56[1]|model = 0.234",
        "set var t566[1]|model = 0.567",
    ]
)

erl.plot(
    "alpha",
    settings=TaoGraphSettings(
        title="My Custom Alpha Plot",
        component="model",
        draw_grid=False,
        x=TaoAxisSettings(
            label="Position - s [m]",
        ),
    ),
)

## Advanced curve settings

`TaoCurveSettings` may be used to customize per-curve settings in simple or gridded plots.

The below example has 4 plots in a 2x2 grid.

Since each plot has a set of curves, we must specify a dictionary for each plot.

That dictionary contains a mapping of `curve_index` (starting with 1) to a `TaoCurveSettings` instance.

See `TaoCurveSettings` for further information on what may be customized.

In [17]:
from pytao.plotting.curves import TaoCurveSettings

erl.plot(
    ["zphase", "zphase", "zphase", "zphase2"],
    grid=(2, 2),
    curves=[
        {1: TaoCurveSettings(ele_ref_name=r"linac.beg\1")},
        {1: TaoCurveSettings(ele_ref_name=r"linac.end\1")},
        {1: TaoCurveSettings(ele_ref_name=r"linac.beg\2")},
        {1: TaoCurveSettings(ele_ref_name=r"linac.end\2")},
    ],
    xlim=(-3, 3),
    ylim=(-10, 10),
    share_x=False,
    include_layout=False,
    width=350, # per plot
    vars=True
)